# Section B - Working with DataFrames and SQL
### B.1 Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.

In [3]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .master("spark://192.168.2.250:7077") \
    .appName("MasudulIslam") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.port", "9999") \
    .config("spark.blockManager.port", "10005") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
    .getOrCreate()

# Initialize Spark Context from the Spark Session
sc = spark.sparkContext

# Path to the CSV file in HDFS
csv_file_path = "hdfs://192.168.2.250:9000/parking-citations.csv"

# Load the CSV into a DataFrame
parking_citations_df = spark.read.option("header", "true").csv(csv_file_path)

# Show the first few rows to verify it's loaded correctly
parking_citations_df.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21T00:00:...|      1251|    NULL|       NULL|            CA|           200304|NULL|HOND|        PA|  

### B.2 Print the schema for the DataFrame.

In [4]:
# Print the schema of the DataFrame
parking_citations_df.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



### B.3 Count the number of rows in the CSV file.

In [5]:
# Count the number of rows in the DataFrame
row_count = parking_citations_df.count()
print(f"The number of rows in the CSV file: {row_count}")

The number of rows in the CSV file: 13077724


### B.4 Count the number of partitions in the underlying RDD

In [6]:
# Count the number of partitions in the underlying RDD
partitions_count = parking_citations_df.rdd.getNumPartitions()
print(f"The number of partitions in the underlying RDD: {partitions_count}")

The number of partitions in the underlying RDD: 16


### B.5 Drop the columns VIN, Latitude and Longitude.

In [7]:
# Drop the specified columns
parking_citations_df = parking_citations_df.drop("VIN", "Latitude", "Longitude")

# Check the schema of the DataFrame to ensure that the VIN, Latitude, and Longitude have been removed or not
parking_citations_df.printSchema()

### B.6 Find the maximum fine amount. How many fines have this amount?

In [12]:
from pyspark.sql.functions import col, max

# Convert the 'Fine amount' column to float
parking_citations_df = parking_citations_df.withColumn("Fine amount", col("Fine amount").cast("float"))

# Find the maximum fine amount
max_fine_amount = parking_citations_df.select(max("Fine amount")).collect()[0][0]
print(f"The maximum fine amount is: {max_fine_amount}")

# Count how many fines have this amount
max_fine_count = parking_citations_df.filter(col("Fine amount") == max_fine_amount).count()
print(f"Number of fines with the maximum amount: {max_fine_count}")

The maximum fine amount is: 1100.0


Number of fines with the maximum amount: 626


### B.7 Show the top 20 most frequent vehicle makes, and their frequencies.

In [13]:
# Group by 'Make' and count the occurrences, then order by count in descending order
top_vehicle_makes = parking_citations_df.groupBy("Make").count().orderBy("count", ascending=False).limit(20)
top_vehicle_makes.show()

+----+-------+
|Make|  count|
+----+-------+
|TOYT|2150768|
|HOND|1479996|
|FORD|1116235|
|NISS| 945133|
|CHEV| 892676|
| BMW| 603092|
|MERZ| 543298|
|VOLK| 432030|
|HYUN| 404917|
|DODG| 391686|
|LEXS| 368420|
| KIA| 328155|
|JEEP| 316300|
|AUDI| 255395|
|MAZD| 242344|
|OTHR| 205546|
| GMC| 184889|
|INFI| 174315|
|CHRY| 159948|
|SUBA| 154640|
+----+-------+



### B.8 Let’s expand some abbreviations in the color column. Create a User Defined Function tocreate a new column, ‘color long’, mapping the original colors to their corresponding values in the dictionary below. If there is no key matching the original color, use the original color.

In [20]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

# Define the full color mapping dictionary
COLORS = {
    'AL': 'Aluminum', 'AM': 'Amber', 'BG': 'Beige', 'BK': 'Black',
    'BL': 'Blue', 'BN': 'Brown', 'BR': 'Brown', 'BZ': 'Bronze',
    'CH': 'Charcoal', 'DK': 'Dark', 'GD': 'Gold', 'GO': 'Gold',
    'GN': 'Green', 'GY': 'Gray', 'GT': 'Granite', 'IV': 'Ivory',
    'LT': 'Light', 'OL': 'Olive', 'OR': 'Orange', 'MR': 'Maroon',
    'PK': 'Pink', 'RD': 'Red', 'RE': 'Red', 'SI': 'Silver', 'SL': 'Silver',
    'SM': 'Smoke', 'TN': 'Tan', 'VT': 'Violet', 'WT': 'White', 'WH': 'White',
    'YL': 'Yellow', 'YE': 'Yellow', 'UN': 'Unknown'
}

# Define a UDF to map color abbreviations to full color names
def expand_color_abbr(color_abbr):
    return COLORS.get(color_abbr, color_abbr)

# Register the UDF with Spark
expand_color_udf = udf(expand_color_abbr, StringType())

# Create the new 'color_long' column using the UDF
parking_citations_df = parking_citations_df.withColumn("ColorLong", expand_color_udf(col("Color")))

# Display the DataFrame to verify the new column
parking_citations_df.select("Color", "ColorLong").show(truncate=False)

+-----+---------+
|Color|ColorLong|
+-----+---------+
|GY   |Gray     |
|WH   |White    |
|BK   |Black    |
|WH   |White    |
|BK   |Black    |
|GY   |Gray     |
|BL   |Blue     |
|BK   |Black    |
|BR   |Brown    |
|SI   |Silver   |
|WH   |White    |
|GO   |Gold     |
|BK   |Black    |
|BK   |Black    |
|BK   |Black    |
|BK   |Black    |
|WH   |White    |
|NULL |NULL     |
|BK   |Black    |
|BK   |Black    |
+-----+---------+
only showing top 20 rows



### B.9 Using this new column, what’s the most frequent colour value for Toyotas (TOYT)?

In [21]:
# Filter for Toyotas, group by the expanded color, and count the occurrences
top_toyota_color = parking_citations_df.filter(col("Make") == "TOYT")\
                                      .groupBy("ColorLong")\
                                      .count()\
                                      .orderBy("count", ascending=False)\
                                      .first()

print(f"The most frequent color for Toyotas is: {top_toyota_color['ColorLong']} with {top_toyota_color['count']} occurrences")

The most frequent color for Toyotas is: Gray with 489697 occurrences
